Install the prerequisites:

In [ ]:
!pip install transformers
!pip install accelerate

Login into HuggingFace, so the program can access the stored models. The Llama series requires additional registration with Meta:

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `myToken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `myToken`


Do the imports:

In [ ]:
import torch
from transformers import AutoTokenizer, pipeline

Define the model identifier:

In [ ]:
#model = "meta-llama/Llama-2-7b-chat-hf"
model = "mistralai/Mistral-7B-v0.1"

Load the tokenizer:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
print(tokenizer)

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


Instantiate text generation pipeline:

In [ ]:
text_gen_pipeline = pipeline(
    task='text-generation',
    model=model,
    top_k=8,
    top_p=0.92,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
print(text_gen_pipeline.model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [ ]:
sequences = text_gen_pipeline(
    text_inputs='What is the most popular food in Asia?',
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=2,
    early_stopping=True
)
for s in sequences:
    print(f"RESULT: {s['generated_text']}")


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


RESULT: What is the most popular food in Asia? This is a question that has been asked by many people, and the answer is not always clear. There are many different types of food that are popular in Asia, and it can be difficult to determine which one is the most popular. In this blog post, we will take a look at some of the most popular foods in Asia and try to determine which one is the most popular.

## What is the most popular food in Asia?

There is no definitive answer to this question
RESULT: What is the most popular food in Asia? This is a question that has been asked by many people, and the answer is not always clear. There are many different types of food that are popular in Asia, and it can be difficult to determine which one is the most popular. In this blog post, we will take a look at some of the most popular foods in Asia and try to determine which one is the most popular. We will also discuss some of the reasons why these foods are so popular.

## What is the most popular

In [ ]:
print(sequences)

[{'generated_text': 'What animals live in the jungle?\n\nThe jungle is home to a wide variety of animals, including mammals, birds, reptiles, amphibians, and insects. Some of the most well-known jungle animals include lions, tigers, elephants, gorillas, monkeys, and snakes.\n\nWhat is the jungle?\n\nThe jungle is a type of forest that is found in tropical regions. It is characterized by its dense vegetation and'}, {'generated_text': 'What animals live in the jungle?\n\nThe jungle is home to a wide variety of animals, including mammals, birds, reptiles, amphibians, and insects. Some of the most well-known jungle animals include lions, tigers, elephants, gorillas, monkeys, and snakes.\n\nWhat is the jungle?\n\nThe jungle is a dense, tropical forest. It is home to a wide variety of plants and animals. The j'}]


In [ ]:
sequences = text_gen_pipeline(
    text_inputs='What is the most popular food in Africa?',
    max_new_tokens=100,
    num_beams=2,
    top_k=2,
    top_p=0.8,
    do_sample=True,
    num_return_sequences=2,
    early_stopping=True
)
for s in sequences:
    print(f"RESULT: {s['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


RESULT: What is the most popular food in Africa?

- The most popular food in Africa is a dish called Jollof rice. It is a rice dish that is made with tomatoes, onions, and peppers.

## What is the most popular food in Africa?

The most popular food in Africa is a dish called Jollof rice. It is a rice dish that is made with tomatoes, onions, and peppers.

## What is the most popular food in Africa
RESULT: What is the most popular food in Africa?

- The most popular food in Africa is a dish called Jollof rice. It is a rice dish that is made with tomatoes, onions, and peppers.

## What is the most popular food in Africa?

The most popular food in Africa is a dish called Jollof rice. It is a rice dish that is made with tomatoes, onions, and peppers.

## What is the most popular food in South
